# Inferring the goals of autonomous agents in Gen.jl

what is Gen.jl (2 sentences)

In [1]:
using Gen

In [13]:
enable_inline()

HTML{String}("<script>\n\tGen = {\n\n    find_choice: function(trace, name) {\n        if (name in trace.recorded) {\n            //console.log(name + \" found in interventions\");\n            return { value: trace.recorded[name], where: \"recorded\" }\n        } else if (name in trace.interventions) {\n            //console.log(name + \" found in interventions\");\n            return { value: trace.interventions[name], where: \"interventions\"}\n        } else if (name in trace.constraints) {\n            //console.log(name + \" found in constraints\");\n            return { value: trace.constraints[name], where: \"constraints\"}\n        } else {\n            return null\n        }\n    },\n\n    register_jupyter_renderer: function(name, render_function) {\n        \n        Jupyter.notebook.kernel.comm_manager.unregister_target(name);\n        Jupyter.notebook.kernel.comm_manager.register_target(name, function(comm, msg) {\n            comm.on_msg(function(msg) {\n                var id = msg.content.data.dom_element_id;\n                var trace = msg.content.data.trace;\n                render_function(id, trace);\n            })\n        });\n        \n    }\n};\n\n</script>")

## 1. Modeling an autonomous agent

Explain ( 2 sentence). Explain that a model is a probabilisttic program, and that @program is used to define a probabilistic program in Gen.jl

In [3]:
include("scene.jl")
include("path_planner.jl")

walk_path (generic function with 1 method)

In [7]:
@program model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
    add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
    add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
    add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
    add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))    
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

We can run the model to generate probable scenarios:. We run a model using `@generate`, which executes a program and populates a trace with the values of the named expressions in the program.

In [8]:
trace = Trace()
@generate(trace, model())
start = value(trace, "start")
destination = value(trace, "destination")
xs = map((i) -> value(trace, "x$i"), 1:4)
ys = map((i) -> value(trace, "y$i"), 1:4)
println("start: ", start)
println("destination: ", destination)
println("xs: ", xs)
println("ys: ", ys)

start: Point(93.7386500434755,37.79116438370564)
destination: Point(88.48257792235299,61.36929086229739)
xs: [93.5184,91.9235,90.5453,89.3539]
ys: [39.0563,47.005,56.8321,62.0805]


If we run it again, we get a different result:

In [9]:
trace = Trace()
@generate(trace, model())
start = value(trace, "start")
destination = value(trace, "destination")
xs = map((i) -> value(trace, "x$i"), 1:4)
ys = map((i) -> value(trace, "y$i"), 1:4)
println("start: ", start)
println("destination: ", destination)
println("xs: ", xs)
println("ys: ", ys)

start: Point(2.307179374751911,68.9543039113342)
destination: Point(43.940174483397264,45.900743483373255)
xs: [1.69084,11.7197,19.1814,32.1906]
ys: [69.4731,73.8802,79.9306,82.2144]


# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing out the values of variables is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. Here is an example trace rendering for our model that uses matplotlib:

In [111]:
javascript"""

function add_wall(paper, scene_group, wall) {
    var rectangle;
    if (wall.orientation == 1) {
        // horizontal wall segment
        rectangle = paper.rect(wall.start.x, wall.start.y, wall.length, wall.thickness);
    } else if (wall.orientation == 2) {
        // vertical wall segment
        rectangle = paper.rect(wall.start.x, wall.start.y, wall.thickness, wall.length);
    }
    scene_group.add(rectangle);
}

function add_tree(paper, scene_group, tree) {
    var side = tree.size;
    var rectangle = paper.rect(tree.center.x - side/2.0, tree.center.y - side/2.0, side, side);
    rectangle.attr({fill: "#3bad29"});
    scene_group.add(rectangle);
}

function render_scene(paper, scene) {
    var group = paper.select("#scene");
    if (group) {
        group.remove();
    }
    group = paper.group();
    group.attr({id: "scene"});
    var obstacles = scene.value.obstacles;
    for (var i=0; i<obstacles.length; i++) {
        var obstacle = obstacles[i];
        switch (obstacle.name) {
            case "Wall":
                add_wall(paper, group, obstacle);
                break;
            case "Tree":
                add_tree(paper, group, obstacle);
                break;
            default:
                break;
        }
    }
}

function render_start(paper, point) {
    var circle = paper.select("#start");
    if (!circle) {
        circle = paper.circle(point.value.x, point.value.y, 3);
        circle.attr({id: "start", fill: "#BE6698", stroke: "#000",
                     "fill-opacity": 1});
    }
    circle.attr({cx: point.value.x, cy: point.value.y, 
                 strokeWidth: point.where == "interventions" ? 4 : 1});
}

function render_destination(paper, point) {
    var circle = paper.select("#destination");
    if (!circle) {
        circle = paper.circle(point.value.x, point.value.y, 3);
        circle.attr({id: "start", fill: "#547F96", stroke: "#000",
                     "fill-opacity": 1});
    }
    circle.attr({cx: point.value.x, cy: point.value.y, 
                 strokeWidth: start.where == "interventions" ? 4 : 1});
}

function render_path(paper, trace) {
    var times = Gen.find_choice(trace, "times");
    var num_path_points = times.value.length;
    for (var i=1; i<=num_path_points; i++) {
        var id = "path-" + i;
        var circle = paper.select("#" + id);
        var x = Gen.find_choice(trace, "x" + i);
        var y = Gen.find_choice(trace, "y" + i);
        if (!circle) {
            circle = paper.circle(x.value, y.value, 3);
            circle.attr({id: id, fill: "#EDD37F", stroke: "#000", "fill-opacity": 0.5});
        }
        circle.attr({cx: x.value, cy: y.value,
                     strokeWidth: (x.where == "interventions" || y.where == "interventions") ? 4 : 1});
    }
}

Gen.register_jupyter_renderer("agent_model_renderer", function(id, trace) {

    var paper = Snap("#" + id);
    console.log("trace: ");
    console.log(trace);

    
    // render the scene elements
    var scene = Gen.find_choice(trace, "scene");
    if (scene) { render_scene(paper, scene); }

    // render start
    var start = Gen.find_choice(trace, "start");
    if (start) { render_start(paper, start); }

    // render destination
    var destination = Gen.find_choice(trace, "destination");
    if (destination) { render_destination(paper, destination); }

    // render points along the path
    render_path(paper, trace);
})
"""

In [112]:
renderer = JupyterInlineRenderer("agent_model_renderer")
inline(renderer)

HTML{String}("<svg id=id_TY8wo9aCRkh32hOl1pr9></svg>")

Let's use this rendering to visualize a run of the our program. It will be visualized in the output of the cell above.

In [113]:
trace = Trace()
@generate(trace, model())
render(renderer, trace)

Since the behavior is stochastic, we need to visualize many samples at once in a grid, to get a sense of the full distribution:

In [41]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 6, 3, 700, 300)
inline(tiled)

HTML{String}("<svg id=id_Hj2RzNf0mIPyzsuXFsWg width=700 height=300 xmlns=\"http://www.w3.org/2000/svg\">\n</svg>\n<script>\n\tvar parent_svg = document.getElementById(\"id_Hj2RzNf0mIPyzsuXFsWg\");\n\tvar tile_width = 116.66666666666667;\n\tvar tile_height = 100.0;\n\tfor (var row=0; row < 3; row++) {\n\t\tfor (var col=0; col < 6; col++) {\n\t\t\tvar id = \"id_Hj2RzNf0mIPyzsuXFsWg_\" + row + \"_\" + col;\n\t\t\tvar x = tile_width * col;\n\t\t\tvar y = tile_height * row;\n\t\t\tvar svg = document.createElementNS(\"http://www.w3.org/2000/svg\", \"svg\");\n\t\t\tsvg.setAttribute(\"id\", id);\n\t\t\tsvg.setAttribute(\"x\", x);\n\t\t\tsvg.setAttribute(\"y\", y);\n\t\t\tsvg.setAttribute(\"width\", tile_width);\n\t\t\tsvg.setAttribute(\"height\", tile_height);\n\t\t\tsvg.setAttribute(\"viewBox\", \"0 0 100 100\");\n\t\t\tparent_svg.appendChild(svg);\n\t\t\tconsole.log(row +\" \" + col + \" \" + id);\n\t\t}\n\t}\n</script>\n")

In [27]:
traces = [begin trace = Trace(); @generate(trace, model()); trace end for i=1:18]
render(tiled, traces)

We can intervene and see what the traces look like when we fix the value of `start` and `destination` to specific values:

In [40]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(tiled)

HTML{String}("<svg id=id_3aqQBAVRbqDFHqFEDWiM width=900 height=300 xmlns=\"http://www.w3.org/2000/svg\">\n</svg>\n<script>\n\tvar parent_svg = document.getElementById(\"id_3aqQBAVRbqDFHqFEDWiM\");\n\tvar tile_width = 100.0;\n\tvar tile_height = 100.0;\n\tfor (var row=0; row < 3; row++) {\n\t\tfor (var col=0; col < 9; col++) {\n\t\t\tvar id = \"id_3aqQBAVRbqDFHqFEDWiM_\" + row + \"_\" + col;\n\t\t\tvar x = tile_width * col;\n\t\t\tvar y = tile_height * row;\n\t\t\tvar svg = document.createElementNS(\"http://www.w3.org/2000/svg\", \"svg\");\n\t\t\tsvg.setAttribute(\"id\", id);\n\t\t\tsvg.setAttribute(\"x\", x);\n\t\t\tsvg.setAttribute(\"y\", y);\n\t\t\tsvg.setAttribute(\"width\", tile_width);\n\t\t\tsvg.setAttribute(\"height\", tile_height);\n\t\t\tsvg.setAttribute(\"viewBox\", \"0 0 100 100\");\n\t\t\tparent_svg.appendChild(svg);\n\t\t\tconsole.log(row +\" \" + col + \" \" + id);\n\t\t}\n\t}\n</script>\n")

In [22]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
intervene!(trace, "destination", Point(90, 90))
traces = Trace[]
for i=1:27
    t = deepcopy(trace)
    @generate(t, model())
    push!(traces, t)
end
render(tiled, traces)

# 3. Probabilistic inference

So far, we have simulated forward from the model, and we have intervened on some random choices and simulated the consequence. However, suppose we had observed a given sequence of locations of the agent, and we wanted to know probable goal locations? This is a query that cannot be answered simply by forward simulation of the program, because the location of the drone is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 15 time points:

In [14]:
observed_x = [0.1, 0.3, 0.5] # TODO..
observed_y = [0.1, 0.3, 0.5] # TODO

3-element Array{Float64,1}:
 0.1
 0.3
 0.5

The first step in inference is to constrain the random choices that are observed.

In [15]:
trace = Trace()
for i=1:15
    constrain!(trace, "x$i", observed_x[i])
    constrain!(trace, "y$i", observed_y[i])
end

LoadError: BoundsError: attempt to access 3-element Array{Float64,1} at index [4]

Now, when we run the program in this trace, we find that the score of the trace tells us how well the trace matches the constraints.

In [16]:
render_grid(renderer, traces)
# TODO render grid should show the score on top of the plot

LoadError: UndefVarError: render_grid not defined

We can use this score to filter out traces that don't match well with the observations. Specifically, we can sample a large number of traces, and pick one in proportion to the exponential of its score. We can then repreat this whole process a number of times:

In [17]:
# show SIR results in a tile plots

# 4. Improving the model

Our model above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that does not match our model's expectations. Let's see what happens when we try to do probabilistic inference in our model, given this data.

In [18]:
# show results (they should look bad)

This is an example of *model mis-specification*, which is when the model is not a good match for the distribution of data we are analyzing.

We can improve the model by adding the possibliltiy thathte agent takes a detour. Specifically, we add the possibility that the agent uses a waypoint and first walks from the starting locatoin to the waypoint and then from the waypoit to the final destination. Let's visualize the resulting samples:

In [19]:
# render_grid()...

Now, we can run inference as before:

In [20]:
# Do SIR with the new dataset and the improved model, and the same number of particles we used above.

In [21]:
# results for the same number of particles as above should look bad.

It looks like the results are not accurate. This is because in the new model, a random forward execution of the imporved model is a lot less likely to match the observations than a random forward exection of the original program. We can try to increase the number of samples to incrase the probability that we get one that matches the data. Note that this will take a few minutes t orun:

In [22]:
# Show results with a larger number of particles (should take < 1 min, should look okay)

This modification of the model mad einference a lot more computationally challenging, and our importance sampling algorihtm is not able to give us real-time inferneces. This motivates the need for a more sophisicated approach to probabilisitc inference.

# 5. Compiling inference with neural networks

There are a number of approaches for creating more efficient inference algorithms. We will focus on one approach, where we train a neural network to make informed guesses about he locatio nof the waypoint. First, let's understand in a bit more detail why the default inference algorihtm was slow

Suppose we knew the right waypoint:

In [23]:
trace = Trace()
constrain!(trace, "use-waypoint", true)
constrain!(trace, "waypoint", Point(0.5, 0.5))

Point(0.5,0.5)

Then the baseline importance sapmling algorithm gives reasonable inferences with fewer samples. We use this idea by training a neural network to make informed guesses about the waypoint, given the observed data as its input. We train the neural network on nany simulatoins of the program. Then, the resulting trained neural network can be used to make informed guesses about the waypooint given any observations.

In [24]:
# show the neural network, and show the training.

Let's visualize the guesses made by the neural network for a few  different datasets:

In [25]:
# show four renderings left to right of different datasets, with circles denoting the neural network's guess about the waypoint. 

Now, let's use this trained neural network to speed up inference:

In [26]:
# show the modififed SIR algorithm, using propose!

In [27]:
# show results for fewer particles, which should be noticeably faster than without the neural network.
# make an explicit comparison.